In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample/submission.csv
/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv
/kaggle/input/neural-network/nn.csv


In [5]:
train_raw = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'

In [6]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test
    
def fill_na(df):
    df = df.sort_values(by=['loc_group', 'startdate']).ffill()
    return df

def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    # df['day_of_week'] = df['startdate'].dt.dayofweek
    # df['week_of_year'] = df['startdate'].dt.isocalendar().week
    return df

def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    train, test = categorical_encode(train, test)

    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]

    return X, y, X_test

In [7]:
X, y, X_test = feature_engineering(train_raw.copy(), test_raw.copy())

In [8]:
# df_train = df_train.drop(['lat', 'lon', 'startdate', 'index'], axis=1)
# df_train = df_train.drop(['startdate', 'index'], axis=1)

In [9]:
# inde = df_test['index']
# df_test = df_test.drop(['lat', 'lon', 'startdate', 'index'], axis=1)
# df_test = df_test.drop(['startdate', 'index'], axis=1)

In [10]:
# le = preprocessing.LabelEncoder()
# le.fit(df_train['climateregions__climateregion'])
# df_train['climateregions__climateregion'] = le.transform(df_train['climateregions__climateregion'])

In [11]:
# df_test['climateregions__climateregion'] = le.transform(df_test['climateregions__climateregion'])

In [12]:
# y = df_train['contest-tmp2m-14d__tmp2m']

In [13]:
# X = df_train.copy()
# X = X.drop('contest-tmp2m-14d__tmp2m', axis=1)
# X = X.fillna(0)

In [14]:
# x_test = df_test.copy()
# x_test = x_test.fillna(0)

# Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression

In [20]:
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.9821323554244852

In [21]:
y_pred = reg.predict(X_test)

In [22]:
y_pred

array([27.920351  , 27.78973238, 27.61175723, ...,  6.22578899,
        6.70665297,  7.32229164])

# SVM

In [ ]:
from sklearn.svm import SVR

In [ ]:
reg1 = SVR(C=1.0, epsilon=0.2)
reg1.fit(X, y)
reg1.score(X, y)

In [ ]:
y_svm = reg1.predict(x_test)

# Random Forest

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [11]:
reg2 = RandomForestRegressor(max_depth=2, random_state=0).fit(X, y)
reg2.score(X, y)

0.8502688791996836

In [12]:
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', None]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

In [14]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
rf_random = RandomizedSearchCV(estimator = reg2, param_distributions = random_grid, 
                               n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X, y)
rf_random.best_params_

In [33]:
reg2 = RandomForestRegressor(n_estimators=80, min_samples_split=2, 
                              min_samples_leaf=4, max_features='log2', 
                              max_depth=20).fit(X, y)
reg2.score(X, y)

0.9986793304251496

In [34]:
pred_rf = reg2.predict(X_test)

In [35]:
pred_rf

array([28.37239093, 28.34709744, 28.12492114, ...,  3.29040493,
        2.61048212,  1.83902939])

# XGBOOST

In [37]:
import xgboost as xgb

In [43]:
reg_xgb = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=100,
                           objective='reg:linear',
                           max_depth=5,
#                            early_stopping_rounds=100,
                           learning_rate=0.01).fit(X, y)
reg_xgb.score(X, y)

[18:23:07] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


0.6479776773887505

In [45]:
pred_xgb = reg_xgb.predict(X_test)

In [46]:
pred_xgb

array([18.17153  , 18.17153  , 18.183956 , ...,  1.2268504,  1.0219326,
        1.0283327], dtype=float32)

# LightGBM

In [47]:
!pip install lightgbm

In [48]:
from lightgbm import LGBMRegressor

In [25]:
_reg = LGBMRegressor()

In [ ]:
boosting_type = ["gbdt", "dart", "goss", "rf"]
num_leaves = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
learning_rate = [float(x) for x in np.arange(0, 1, 0.01)]
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 2000, num = 10)]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'boosting_type': boosting_type,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'num_leaves': num_leaves}

In [ ]:
rf_random = RandomizedSearchCV(estimator = _reg, param_distributions = random_grid, 
                               n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X, y)
rf_random.best_params_

In [49]:
reg_lgb = LGBMRegressor(
boosting_type='gbdt',
objective='regression',
metric='rmse',
max_depth= 4,
num_leaves= 31,
learning_rate= 0.05,
feature_fraction= 0.9,
bagging_fraction= 0.8,
bagging_freq= 5).fit(X, y)
reg_lgb.score(X, y)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


0.9786215275017576

In [52]:
pred_lgb = reg_lgb.predict(X_test)

In [53]:
pred_lgb

array([28.06983023, 28.01902323, 28.08151286, ...,  3.37707497,
        3.01862025,  3.13402633])

# CatBoost

In [54]:
!pip install catboost

In [55]:
from catboost import CatBoostRegressor, Pool

In [30]:
_reg = CatBoostRegressor()

In [31]:
learning_rate = [float(x) for x in np.arange(0.5, 0.7, 0.001)]
depth = [int(x) for x in np.linspace(0, 20, num = 2)]
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 2000, num = 10)]

In [32]:
random_grid = {'depth': depth,
               'learning_rate': learning_rate,
               'n_estimators': n_estimators}

In [34]:
rf_random = RandomizedSearchCV(estimator = _reg, param_distributions = random_grid, 
                               n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X, y)
rf_random.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END depth=20, learning_rate=0.6170000000000001, n_estimators=933; total time=   0.7s
[CV] END depth=20, learning_rate=0.6170000000000001, n_estimators=933; total time=   0.7s
[CV] END depth=20, learning_rate=0.6500000000000001, n_estimators=1573; total time=   0.6s
0:	learn: 9.4339775	total: 79.4ms	remaining: 6.27s
1:	learn: 9.4339775	total: 90ms	remaining: 3.51s
2:	learn: 9.4339775	total: 99.6ms	remaining: 2.56s
3:	learn: 9.4339775	total: 117ms	remaining: 2.22s
4:	learn: 9.4339775	total: 142ms	remaining: 2.12s
5:	learn: 9.4339775	total: 162ms	remaining: 2s
6:	learn: 9.4339775	total: 177ms	remaining: 1.85s
7:	learn: 9.4339775	total: 197ms	remaining: 1.77s
8:	learn: 9.4339775	total: 213ms	remaining: 1.68s
9:	learn: 9.4339775	total: 232ms	remaining: 1.63s
10:	learn: 9.4339775	total: 254ms	remaining: 1.59s
11:	learn: 9.4339775	total: 273ms	remaining: 1.55s
12:	learn: 9.4339775	total: 291ms	remaining: 1.5s
13:	learn: 9.433977

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/catboost/core.py", line 5733, in fit
    save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
  File "/opt/conda/lib/python3.7/site-packages/catboost/core.py", line 2347, in _fit
    callbacks=callbacks
  File "/opt/conda/lib/python3.

0:	learn: 9.8678876	total: 71.2ms	remaining: 5.63s
1:	learn: 9.8678876	total: 87.9ms	remaining: 3.43s
2:	learn: 9.8678876	total: 104ms	remaining: 2.67s
3:	learn: 9.8678876	total: 119ms	remaining: 2.26s
4:	learn: 9.8678876	total: 134ms	remaining: 2s
5:	learn: 9.8678876	total: 149ms	remaining: 1.83s
6:	learn: 9.8678876	total: 163ms	remaining: 1.7s
7:	learn: 9.8678876	total: 178ms	remaining: 1.6s
8:	learn: 9.8678876	total: 194ms	remaining: 1.53s
9:	learn: 9.8678876	total: 209ms	remaining: 1.46s
10:	learn: 9.8678876	total: 223ms	remaining: 1.4s
11:	learn: 9.8678876	total: 238ms	remaining: 1.35s
12:	learn: 9.8678876	total: 254ms	remaining: 1.31s
13:	learn: 9.8678876	total: 269ms	remaining: 1.27s
14:	learn: 9.8678876	total: 292ms	remaining: 1.26s
15:	learn: 9.8678876	total: 309ms	remaining: 1.24s
16:	learn: 9.8678876	total: 324ms	remaining: 1.2s
17:	learn: 9.8678876	total: 339ms	remaining: 1.17s
18:	learn: 9.8678876	total: 354ms	remaining: 1.14s
19:	learn: 9.8678876	total: 369ms	remaining: 1

{'n_estimators': 80, 'learning_rate': 0.5860000000000001, 'depth': 0}

In [35]:
reg4 = CatBoostRegressor(n_estimators=80, learning_rate=0.5860000000000001, depth=0).fit(X, y)
reg4.score(X, y)

0:	learn: 9.8678876	total: 27.6ms	remaining: 2.18s
1:	learn: 9.8678876	total: 46.5ms	remaining: 1.81s
2:	learn: 9.8678876	total: 63.1ms	remaining: 1.62s
3:	learn: 9.8678876	total: 79.9ms	remaining: 1.52s
4:	learn: 9.8678876	total: 96.7ms	remaining: 1.45s
5:	learn: 9.8678876	total: 114ms	remaining: 1.4s
6:	learn: 9.8678876	total: 130ms	remaining: 1.36s
7:	learn: 9.8678876	total: 146ms	remaining: 1.31s
8:	learn: 9.8678876	total: 162ms	remaining: 1.28s
9:	learn: 9.8678876	total: 178ms	remaining: 1.24s
10:	learn: 9.8678876	total: 194ms	remaining: 1.21s
11:	learn: 9.8678876	total: 210ms	remaining: 1.19s
12:	learn: 9.8678876	total: 227ms	remaining: 1.17s
13:	learn: 9.8678876	total: 253ms	remaining: 1.19s
14:	learn: 9.8678876	total: 269ms	remaining: 1.17s
15:	learn: 9.8678876	total: 284ms	remaining: 1.14s
16:	learn: 9.8678876	total: 299ms	remaining: 1.11s
17:	learn: 9.8678876	total: 315ms	remaining: 1.08s
18:	learn: 9.8678876	total: 330ms	remaining: 1.06s
19:	learn: 9.8678876	total: 346ms	rem

2.220446049250313e-16

In [ ]:
reg = CatBoostRegressor(
    verbose = 0,
    learning_rate = 0.0980689972639084,
    subsample = 0.800000011920929, 
    l2_leaf_reg = 4,
    max_depth = 7,
    random_state = 1212,
    loss_function='RMSE',
    model_size_reg = 0.5).fit(X, y)
reg.score(X, y)

In [ ]:
pred_cb = reg.predict(X_test)

In [ ]:
pred_cb

# Neural Networks

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=5).fit(X, y)

In [ ]:
ypred = regr.predict(x_test)

In [47]:
sub = pd.read_csv("/kaggle/input/sample/submission.csv")

In [48]:
y_sub = sub["contest-tmp2m-14d__tmp2m"]

In [49]:
y_sub

0        28.681589
1        28.697912
2        28.656298
3        28.718031
4        28.753995
           ...    
31349     6.478418
31350     6.297699
31351     5.542424
31352     6.070770
31353     6.184927
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64

In [42]:
pred_nn = pd.read_csv("/kaggle/input/neural-network/nn.csv")

In [43]:
y_nn = pred_nn["contest-tmp2m-14d__tmp2m"]

In [44]:
y_nn

0        30.290146
1        30.711065
2        31.173037
3        31.525070
4        31.708550
           ...    
31349     4.624471
31350     4.717812
31351     4.407266
31352     4.804049
31353     5.016165
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64

In [50]:
y_pred

array([27.920351  , 27.78973238, 27.61175723, ...,  6.22578899,
        6.70665297,  7.32229164])

In [51]:
pred_lgb

array([28.06983023, 28.01902323, 28.08151286, ...,  3.37707497,
        3.01862025,  3.13402633])

In [52]:
pred_rf

array([28.79462096, 28.69543032, 28.43102367, ...,  2.13786614,
        1.95887785,  1.98686304])

In [53]:
pred_cb

array([28.75599089, 28.88941132, 28.87249032, ...,  2.9909791 ,
        3.37797168,  3.48452787])

In [3]:
pred = (35*y_sub + 3*y_pred + pred_lgb + 1.25*pred_rf + pred_cb + y_nn)/42.25

NameError: name 'y_sub' is not defined

In [ ]:
pred

In [186]:
submit = pd.DataFrame()

In [187]:
submit['contest-tmp2m-14d__tmp2m'] = pred
submit['index'] = inde

In [188]:
submit

,contest-tmp2m-14d__tmp2m,index
0,28.662272,375734
1,28.682180,375735
2,28.650070,375736
3,28.720707,375737
4,28.761703,375738
...,...,...
31349,6.310793,407083
31350,6.110728,407084
31351,5.377938,407085
31352,5.887747,407086


In [159]:
submit.to_csv('en5.csv', index = False)